In [1]:
import pandas as pd 
import numpy as np

# Test Dataset 

In [2]:
df = pd.read_csv("../3_regClassif/senti_clientwise_try1.csv")

In [4]:
print(df.shape)
df.head()

(59094, 6)


Unnamed: 0  OrderId                                           CliChats  \
0           0    17393  I only rented this because i loved the first m...   
1           1    17393  'I don't understand. None of this makes any se...   
2           2    17393  I know that to include everything in the book,...   
3           3    17393  This is easily the worst, most offensive piece...   
4           4    17394  When you see Barry Corbin in the cast of a mov...   

                                             SpChats  CliAvgSenti  \
0  This film plunges headlong into the realm of t...          0.4   
1  Taylor Hackford wanted to make this movie for ...          0.4   
2  I don't know about the real Cobb but I got the...          0.8   
3  The film starts in the Long Island Kennel Club...          0.4   
4  Richard Attenborough who already given us magn...          0.4   

   Sp_avg_senti  
0           0.4  
1           0.6  
2           0.0  
3           0.6  
4           0.6

# 1.Text Preprocessing

In [7]:
# lowercasing
df['CliChats']= df['CliChats'].str.lower()
df['SpChats']=df['SpChats'].str.lower()

In [8]:
df.head(2)

Unnamed: 0  OrderId                                           CliChats  \
0           0    17393  i only rented this because i loved the first m...   
1           1    17393  'i don't understand. none of this makes any se...   

                                             SpChats  CliAvgSenti  \
0  this film plunges headlong into the realm of t...          0.4   
1  taylor hackford wanted to make this movie for ...          0.4   

   Sp_avg_senti  
0           0.4  
1           0.6

In [9]:
#removing html tags 
import re 
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'',text)

In [10]:
df['CliChats'] = df['CliChats'].apply(remove_html_tags)
df['SpChats'] = df['SpChats'].apply(remove_html_tags)

In [11]:
df.head(2)

Unnamed: 0  OrderId                                           CliChats  \
0           0    17393  i only rented this because i loved the first m...   
1           1    17393  'i don't understand. none of this makes any se...   

                                             SpChats  CliAvgSenti  \
0  this film plunges headlong into the realm of t...          0.4   
1  taylor hackford wanted to make this movie for ...          0.4   

   Sp_avg_senti  
0           0.4  
1           0.6

In [12]:
#remove URLs
def remove_url(text):
    pattern = re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [14]:
df['CliChats'] = df['CliChats'].apply(remove_url)
df['SpChats'] = df['SpChats'].apply(remove_url)

In [15]:
df.head(2)

Unnamed: 0  OrderId                                           CliChats  \
0           0    17393  i only rented this because i loved the first m...   
1           1    17393  'i don't understand. none of this makes any se...   

                                             SpChats  CliAvgSenti  \
0  this film plunges headlong into the realm of t...          0.4   
1  taylor hackford wanted to make this movie for ...          0.4   

   Sp_avg_senti  
0           0.4  
1           0.6

In [22]:
#remove punctuations 
def remove_punctuations(text):
    return re.sub(r'[^\w\s]', '', text)

In [23]:
df['CliChats'] = df['CliChats'].apply(remove_punctuations)
df['SpChats'] = df['SpChats'].apply(remove_punctuations)

In [24]:
df.head(2)

Unnamed: 0  OrderId                                           CliChats  \
0           0    17393  i only rented this because i loved the first m...   
1           1    17393  i dont understand none of this makes any sense...   

                                             SpChats  CliAvgSenti  \
0  this film plunges headlong into the realm of t...          0.4   
1  taylor hackford wanted to make this movie for ...          0.4   

   Sp_avg_senti  
0           0.4  
1           0.6

In [19]:
# chat word treatment
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mfw" : "my face when",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}
def chat_word_treatment(text):
    new_text = []
    for w in text.split():
        if w.lower() in abbreviations:
            new_text.append(abbreviations[w.lower()])
        else :
            new_text.append(w)
    return " ".join(new_text)

In [20]:
df['CliChats'] = df['CliChats'].apply(chat_word_treatment)
df['SpChats'] = df['SpChats'].apply(chat_word_treatment)

In [21]:
df.head(2)

Unnamed: 0  OrderId                                           CliChats  \
0           0    17393  i only rented this because i loved the first m...   
1           1    17393  'i don't understand. none of this makes any se...   

                                             SpChats  CliAvgSenti  \
0  this film plunges headlong into the realm of t...          0.4   
1  taylor hackford wanted to make this movie for ...          0.4   

   Sp_avg_senti  
0           0.4  
1           0.6

In [26]:
#spelling correction
from textblob import TextBlob
def spell_correct(text):
    new_text = []
    for w in text.split():
        textBlb = TextBlob(w)
        new_text.append(textBlb.correct().string)
    return " ".join(new_text)

In [27]:
df['CliChats'] = df['CliChats'].apply(spell_correct)
df['SpChats'] = df['SpChats'].apply(spell_correct)

KeyboardInterrupt: 

In [ ]:
df.head(2)

In [28]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/neelchoksi19bce0990/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [29]:
# remove stopwords:
def remove_stopwords(text):
    new_text=[]
    
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else :
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [ ]:
df['CliChats'] = df['CliChats'].apply(remove_stopwords)
df['SpChats'] = df['SpChats'].apply(remove_stopwords)

In [ ]:
df.head(2)

In [ ]:
#stemming:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [ ]:
df['CliChats'] = df['CliChats'].apply(stem_words)
df['SpChats'] = df['SpChats'].apply(stem_words)

In [ ]:
df.head(2)

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
sent1 = "I ada ad fa dfa df!"
word_tokenize(sent1)

# 2.Text Representation

# 3.Word2Vec

# 4.TextClassif - ML Models
- text vectorization : BOW, NGrams, TFIDF , word2vec
- NB
- Random forest
- hyperparamter tuning : Email Spam classifier vid 

# 5.own prev- hybrid ml models apply 
- last part use : https://github.com/campusx-official/sms-spam-classifier/blob/main/sms-spam-detection.ipynb

# 6.pydata - london : ANN , LSTM , GRU 

# 7.pytorch - ensemble use on ANN , LSTM , GRU